In [196]:
import os
import sys
from tqdm.auto import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time

from IPython.display import clear_output

sys.path.append('src')
from train_test import train_test_split

from metrics import compute_metrics

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATA_PATH = './data.nosync/'

In [4]:
# Load data files
train = pd.read_csv(DATA_PATH + 'train.csv')
valid = pd.read_csv(DATA_PATH + 'valid.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
lastfm_360_behav = pd.read_csv(DATA_PATH + 'behav-360k-processed.csv')
lastfm_360_demo = pd.read_csv(DATA_PATH + 'demo-360k-processed.csv')
lastfm_360_demo = lastfm_360_demo.set_index('user_email').astype(bool).reset_index()
test_users = np.load(DATA_PATH + 'test_users.npy')

In [5]:
# Rescale data
train_plays_max = train['log_plays'].max()
train['log_plays'] = train['log_plays'] / train_plays_max
valid['log_plays'] = valid['log_plays'] / train_plays_max
test['log_plays'] = test['log_plays'] / train_plays_max

In [14]:
train

,user_email,artist_id,log_plays
0,184649,460,0.000000
1,255115,2495,0.424939
2,113122,250,0.272390
3,249162,2043,0.541921
4,255027,603,0.160624
...,...,...,...
14049672,21192,3149,0.355721
14049673,219208,12864,0.357251
14049674,223801,28,0.297401
14049675,21038,29798,0.312301


In [197]:
def get_users_for_artist(df, artist_id:int):
    return df[df['artist_id'] == artist_id]['user_email'].values

In [198]:
def get_correlation_list(df, user_email: str, selected_users:list, filter_artists=True, verbose=False):
    
    # Filter the behavioral data for the listed users
    selected_behav = df[(df['user_email'].isin(selected_users)) | (df['user_email'] == user_email)]
    
    # Make sure there is no duplicated rows for given user and artist pair
    selected_behav = selected_behav.groupby(['user_email', 'artist_id'], as_index=False).sum()
    
    # Compute the number of artists listened in the selected group
    selected_artists = selected_behav['artist_id'].value_counts()#.index.values
    if verbose: print(f"Total artists: {len(selected_artists.index.values)}")
    
    # Filter the artist to reduce the dimention
    if filter_artists:
        # Remove all artist that have been listened by less than 1% of the user list. 
        artist_threshold = len(selected_users) / 100
        selected_artists = selected_artists[selected_artists > artist_threshold].index.values
        selected_behav = selected_behav[selected_behav['artist_id'].isin(selected_artists)]
        
    # Build the user raking dataset (user_email index and one column per artist)
    user_scores = selected_behav.pivot(index='user_email', columns='artist_id', values='log_plays').fillna(0)
    
    # Compute correlation between users
    correlation_list = user_scores.corrwith(user_scores.loc[user_email], axis=1)
    
    return correlation_list, selected_behav

In [199]:
def compute_prediction(data, user_email: str, artist_id: int, selected_corr: np.array, selected_behav: pd.DataFrame, verbose=False):
    # Get the behav data for the remaining users
    selected_behav = selected_behav[selected_behav['user_email'].isin(list(selected_corr.index.values))]
    #final_behav = selected_behav
    # Compute the normalized average number of play
    avg_log_plays = selected_behav.groupby('user_email')['log_plays'].mean()
    
    # Get the log play for a given artist
    artist_log_plays = selected_behav[(selected_behav['artist_id'] == artist_id)]\
                                    [['user_email', 'log_plays']].set_index('user_email')['log_plays']
    
    # Compute the plays variance from the similar users
    var_pred = (selected_corr * (artist_log_plays - avg_log_plays)).sum() / selected_corr.sum()
    # Compute user total number of play and average number of play
    user_log_plays = data[data['user_email'] == user_email]['log_plays']
    
    if verbose: print(f'user plays: {user_log_plays}, mean: {user_log_plays.mean()}')
    
    # Compute the estimated number of play for new artist (average + normalized var * number of plays)
    predicted_plays = user_log_plays.mean() + var_pred 
    
    return predicted_plays

In [200]:
def get_artist_prediction(data, user_email:str, artist_id:int, corr_threshold=0.2, 
                          filter_demo=False, filter_artist=True, verbose=False):
    start = time.time()
    
    # Select users that have already listened to the given artist 
    selected_users = get_users_for_artist(data, artist_id)
    if verbose: print(f"Selected users: {len(selected_users)}")
    
    # Compute correlation with selected users
    start_corr = time.time()
    correlation_list, selected_behav = get_correlation_list(data, user_email, selected_users, filter_artist)
    end_corr = time.time()
    if verbose: print(f"Correlation list computation time: {end_corr - start_corr}")
    
    # Compute prediction
    selected_corr = correlation_list[correlation_list > corr_threshold]
    selected_corr = selected_corr.drop(user_email)
    
    
    if verbose: print(f"Selected users (after correlation): {len(selected_corr)}")
    
    prediction = compute_prediction(data, user_email, artist_id, selected_corr, selected_behav)
    end = time.time()
    
    if verbose: print(f"Total time for prediction: {end - start}")
    return prediction
    

In [201]:
seen_artists = train.append(valid).drop(columns='log_plays')
seen_artists = seen_artists[seen_artists['user_email'].isin(test_users)]
seen_artists = seen_artists.groupby('artist_id').first().index.values

In [281]:
def pred_func(data, user_id, artists: list):
    preds = []
    for a in artists:
        pred = get_artist_prediction(data, user_id, a)
        if np.isnan(pred):
            pred = 0
        preds.append(pred)
    return preds

In [256]:
def top_100_artists(data):
    total_log_plays = data.groupby('artist_id').sum()
    ranked_artist_scores = total_log_plays.sort_values(['log_plays'], ascending=False)['log_plays']
    return ranked_artist_scores[:100].index.values

In [257]:
top_100 = top_100_artists(train)

In [258]:
def get_ratings(train, full_train, artists):
    ratings = {}
    top_100 = set(top_100_artists(train))
    set_artists = set(artists)
    for user, user_df in tqdm(train.groupby('user_email')):
        new_artists = np.array(list(top_100.intersection(set_artists - set(user_df['artist_id']))))
        ratings[user] = [new_artists, pred_func(full_train, user, new_artists)]
        print(ratings[user])
    return ratings

In [259]:
train_subset = train[train['user_email'].isin(test_users[:100])]

In [260]:
pred_ratings = get_ratings(train_subset, train,seen_artists)

KeyboardInterrupt: 

In [ ]:

# Evaluate the model 
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return mean_squared_error(y_actual, y_predicted, squared=False)

In [282]:
test_small = test[test['user_email'].isin(test_users[:100])]

In [ ]:
preds = {}
true = {}
for (user, user_df) in tqdm(test_small.groupby('user_email')):
    artists = user_df['artist_id'].values
    preds[user] = [artists, pred_func(train, user, artists)]
    print(preds[user])
    true[user] = [artists, user_df['log_plays'].values]

[array([ 2020, 13632,  2702, 10711,   245]), [0.33557867316222717, 0.3526058388926968, 0.3539373601870344, 0.3402115569069563, 0.35614043450511984]]


/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[array([ 1162, 41587,  3702, 17090, 17228,  5695]), [0.41079650615218655, 0, 0.4173157132237809, 0.45793798410229236, 0.409280277834704, 0.4034456657319745]]
[array([  434,  4276,  1150,   677,  4511, 39918]), [0.5887299651827055, 0.5681708310717594, 0.5565459109966965, 0.56261092393758, 0.549132315634738, 0.5396215452303871]]
[array([2700, 1475,  247, 1408, 3211,  264]), [0.20625935284124935, 0.21713938655364914, 0.22595995092604582, 0.2203302808267392, 0.24282669429611822, 0.22210110950643897]]
[array([14578,  5012,  1680,   731,    62]), [0.25515684892290424, 0.28066864944893455, 0.28149018575695417, 0.28201935049612004, 0.3143343348328163]]
[array([ 4297, 19540,   268,  5416,  3524,    63]), [0.4150727586775414, 0.39603835470462184, 0.4052052533275835, 0.40152985421877785, 0.39247197969342434, 0.4232941552859127]]
[array([   62,   478, 10235,   731,  1156,   350]), [0.38766746645149036, 0.4123297082050091, 0.3632991936415807, 0.3672074262490962, 0.3555363220571916, 0.38022766894572

/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[array([ 3279, 32332, 37497,   437,  3542, 22272]), [0.1801519596305229, 0, 0, 0.19619419957691464, 0.186591703778344, 0.15966698226250203]]


...

[array([ 1157,  8828,  1203,   264,  4235, 42197]), [0.4501969456847036, 0.4295431869256831, 0.46020382380660946, 0.4268855740534707, 0.43840713214217664, 0]]
[array([7487,    5,  146,  688, 2433]), [0.35366343756731994, 0.3179596112085471, 0.33981957384286965, 0.3157935806335297, 0.30811320280857]]


/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[array([ 4589,   562, 37801,  3729,  2141,    62]), [0, 0, 0, 0.37916744534982205, 0.35459036063837035, 0.3515481411234305]]
[array([4810, 2017, 2817,  709, 1041]), [0.39389307658930595, 0.4083326811869151, 0.41881406579894537, 0.4150984078793912, 0.41304167352297555]]
[array([   13,  1689, 20972,   885,   909]), [0.33193376719876017, 0.32740824076523356, 0.3636474872291857, 0.3508392094333046, 0.3698822015247538]]


/Users/pierre/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[array([2697,   55, 5706, 2119,  552]), [0.35899472189907117, 0.40098454025268965, 0, 0.3999859878215652, 0.4004257010459638]]
